In [ ]:
#  The MIT License (MIT)
#
#  Copyright (c) 2015-2022 Advanced Micro Devices, Inc. All rights reserved.
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the 'Software'), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in
#  all copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED 'AS IS', WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL THE
#  AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#  LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#  OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
#  THE SOFTWARE.


# NFNet Inference with AMD MIGraphX


Normalizer-Free ResNet is a new residual convolutional network providing new state-of-the-art Top-1 accuracy of 86.5% at ImageNet dataset. The most important feature of the model is removing batch normalization. Instead of batch normalization, it uses adaptive gradient clipping to provide same regularization effect of BatchNorm. <br> Details of this network: https://arxiv.org/abs/2102.06171

In this notebook, we are showing: <br>
- How to optimize NFNet ONNX model with AMD MIGraphX.
- How to run inference on AMD GPU with the optimized ONNX model.

The NFNet utilized in this example is the smallest NFNet version, F0: 71.5M parameters (83.6% top-1 accuracy on ImageNet)

Please make sure MIGraphX Python API is installed following the instructions at Github page.

## Requirements

In [ ]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y 

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements_nfnet.txt

In [ ]:
import numpy as np
import cv2
import json
from PIL import Image
import time
from os import path 

### Importing AMD MIGraphX Python Module

In [ ]:
import migraphx

### Create NFNet ONNX file
Following repository provides functionality to create NFNet ONNX file from PyTorch model.

In [ ]:
!wget -nc https://www.dropbox.com/s/u4ga8zyxtppfzxc/dm_nfnet_f0.onnx

### Load ImageNet labels

In [ ]:
with open('../python_resnet50/imagenet_simple_labels.json') as json_data:
    labels = json.load(json_data)

In [ ]:
## Load ONNX model using MIGraphX

In [ ]:
model = migraphx.parse_onnx("dm_nfnet_f0.onnx")
model.compile(migraphx.get_target("gpu"))

print(model.get_parameter_names())
print(model.get_parameter_shapes())
print(model.get_output_shapes())

### Functions for image processing

In [ ]:
def make_nxn(image, n):
    height, width = image.shape[:2]    
    if height > width:
        dif = height - width
        bar = dif // 2 
        square = image[(bar + (dif % 2)):(height - bar),:]
        return cv2.resize(square, (n, n))
    elif width > height:
        dif = width - height
        bar = dif // 2
        square = image[:,(bar + (dif % 2)):(width - bar)]
        return cv2.resize(square, (n, n))
    else:
        return cv2.resize(image, (n, n))
    
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):  
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def input_process(frame, dim):
    # Crop and resize original image
    cropped = make_nxn(frame, dim)
    # Convert from HWC to CHW
    chw = cropped.transpose(2,0,1)
    # Apply normalization
    pp = preprocess(chw)
    # Add singleton dimension (CHW to NCHW)
    data = np.expand_dims(pp.astype('float32'),0)
    return data

### Download example image

In [ ]:
# Fetch example image: traffic light
!wget -nc http://farm5.static.flickr.com/4072/4462811418_8bc2bd42ca_z_d.jpg -O traffic_light.jpg
# Read the image
im = cv2.imread('traffic_light.jpg')

In [ ]:
# Process the read image to conform input requirements
data_input = input_process(im, 192)

# Run the model
start = time.time()
results = model.run({'inputs':data_input}) # Your first inference would take longer than the following ones.
print(f"Time inference took: {1000*(time.time() - start):.2f}ms")
# Extract the index of the top prediction
res_npa = np.array(results[0])
print(f"\nResult: {labels[np.argmax(res_npa)]}")

In [ ]:
# Run the model again, first one would take long
start = time.time()
results = model.run({'inputs':data_input}) # Your first inference would take longer than the following ones.
print(f"Time inference took: {1000*(time.time() - start):.2f}ms")
# Extract the index of the top prediction
res_npa = np.array(results[0])
print(f"\nResult: {labels[np.argmax(res_npa)]}")